In [1]:
import pandas as pd
import numpy as np
from sql_functions import *
import plotly.express as px
import json
from normal_round import normal_round as normal_round

In [2]:
pd.set_option('display.max_rows', 6000)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', None)

In [3]:
df_laender_abs = get_dataframe('SELECT * FROM public.laender_jugend_tat_absolut_2022_until_2018;')
df_laender_rel = get_dataframe('SELECT * FROM public.laender_jugend_tat_relativ_2022_until_2018;')

In [4]:
df_laender_abs.head(5)

,schluessel,straftat,bundesland,sexus,tatverdaechtige_insgesamt,kinder_10_bis_unter_12,kinder_12_bis_unter_14,kinder_10_bis_unter_14,kinder_unter_14,jugendliche_14_bis_unter_16,jugendliche_16_bis_unter_18,jugendliche_14<18,heranwachsende_18_bis_unter_21,jugendl_u_heranwachsende_14_bis_unter_21,erwachsene_ab_21,year
0,------,Straftaten insgesamt,Baden-Württemberg,M,184801,1305,2975,4280,5246,6585,10078,16663,20122,36785,142770,2018
1,------,Straftaten insgesamt,Bayern,M,226208,1390,3101,4491,6276,6786,10604,17390,24065,41455,178477,2018
2,------,Straftaten insgesamt,Berlin,M,101377,831,1561,2392,3112,2881,3977,6858,7941,14799,83466,2018
3,------,Straftaten insgesamt,Brandenburg,M,49670,547,962,1509,2050,1869,2469,4338,3910,8248,39372,2018
4,------,Straftaten insgesamt,Bremen,M,18519,95,224,319,389,642,1125,1767,1934,3701,14429,2018


In [5]:
#df_laender_rel.tail()

### Calculate most occuring crime per state - rank and percentage of total

In [6]:
df_1 = df_laender_abs.copy()
#df_1 = df_1[df_1['sexus'] == 'X']
df_1 = df_1[['schluessel', 'straftat', 'bundesland', 'sexus', 'year', 'jugendliche_16_bis_unter_18']]
df_1 = df_1.rename(columns={'jugendliche_16_bis_unter_18': 'offenders'})

df_2 = df_1[df_1['schluessel'] == '------']
df_2 = df_2[['bundesland', 'sexus', 'year', 'offenders']]
df_laender_straftaten_insg_absolut = df_2.copy() # needed later

df_3 = df_1[df_1['schluessel'] != '------']

df_3['rank'] = (
    df_3.groupby(['bundesland', 'sexus', 'year'])['offenders']
        .rank(ascending=False, method='first')
        .astype(int)
)

df_4 = df_3.merge(df_2, on=['bundesland', 'sexus', 'year'], how='left')

df_4['certain_crime_percent_of_total_crime'] = df_4['offenders_x'] / df_4['offenders_y'] * 100
df_4 = df_4.drop(columns='offenders_y')
df_4 = df_4.reset_index()
df_4 = df_4.drop(columns='index')

values = df_4['certain_crime_percent_of_total_crime'].tolist()
new_values = []
for v in values:
    new_v = normal_round(num=v, ndigits=2)
    new_values.append(new_v)
    
df_4['certain_crime_percent_of_total_crime'] = new_values

df_4 = df_4[['schluessel', 'bundesland', 'sexus', 'year', 'straftat', 'rank', 'certain_crime_percent_of_total_crime']]

crime_rank_and_percent_of_total_per_state = df_4.copy()
crime_rank_and_percent_of_total_per_state.head()

/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_14752/1807079327.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3['rank'] = (


,schluessel,bundesland,sexus,year,straftat,rank,certain_crime_percent_of_total_crime
0,100000,Baden-Württemberg,M,2018,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,5,3.15
1,100000,Bayern,M,2018,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,5,3.27
2,100000,Berlin,M,2018,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,6,3.04
3,100000,Brandenburg,M,2018,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,5,3.65
4,100000,Bremen,M,2018,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,6,1.69


### Create overview - number of offenders per state - absolut and relativ

In [7]:
df_10 = df_laender_rel.copy()
df_10 = df_10.rename(columns={'jugendliche_16_bis_unter_18_rel': 'offenders_rel'})
df_10 = df_10[(df_10['schluessel'] == '------')]

jugendkrim_insg_rel = df_10['offenders_rel'].tolist()
jugendkrim_insg_rel_rounded = []
for v in jugendkrim_insg_rel:
    v_new = int(normal_round(v, 0))
    jugendkrim_insg_rel_rounded.append(v_new)
df_10['offenders_rel'] = jugendkrim_insg_rel_rounded

df_10 = df_10[['bundesland', 'sexus', 'year', 'offenders_rel']]
df_10 = df_10.reset_index()
df_10 = df_10.drop(columns='index')

df_11 = df_laender_straftaten_insg_absolut.merge(df_10, on=['bundesland', 'sexus', 'year'], how='left')
df_laender_straftaten_insg_abs_u_rel = df_11.copy()
df_laender_straftaten_insg_abs_u_rel.head()

,bundesland,sexus,year,offenders,offenders_rel
0,Baden-Württemberg,M,2018,10078,8600
1,Bayern,M,2018,10604,7998
2,Berlin,M,2018,3977,13427
3,Brandenburg,M,2018,2469,11170
4,Bremen,M,2018,1125,16922


### Merge crimes that <br>rank on 1, 2, and 3 <br>and <br>overall number of crimes - absolut and relativ <br>to every state for every year (->one DataFrame)

In [8]:
df_20 = crime_rank_and_percent_of_total_per_state.copy()
df_rank_1 = df_20[df_20['rank'] == 1]
df_rank_2 = df_20[df_20['rank'] == 2]
df_rank_3 = df_20[df_20['rank'] == 3]
df_rank_4 = df_20[df_20['rank'] == 4]
df_rank_5 = df_20[df_20['rank'] == 5]
df_rank_1.head()

,schluessel,bundesland,sexus,year,straftat,rank,certain_crime_percent_of_total_crime
98,220000,Berlin,M,2018,"Körperverletzung §§ 223-227, 229, 231 StGB",1,23.21
100,220000,Bremen,M,2018,"Körperverletzung §§ 223-227, 229, 231 StGB",1,19.29
105,220000,Nordrhein-Westfalen,M,2018,"Körperverletzung §§ 223-227, 229, 231 StGB",1,27.50
107,220000,Saarland,M,2018,"Körperverletzung §§ 223-227, 229, 231 StGB",1,26.33
110,220000,Schleswig-Holstein,M,2018,"Körperverletzung §§ 223-227, 229, 231 StGB",1,24.90


In [9]:
df_30 = df_laender_straftaten_insg_abs_u_rel.copy()

df_31 = df_30.merge(df_rank_1, on=['bundesland', 'sexus', 'year'], how='left')
df_31 = df_31.rename(columns={'straftat': 'crimes_on_rank_1',
                               'certain_crime_percent_of_total_crime': 'percentage_of_rank_1_on_crime_total',
                               'schluessel': 'schluessel_crimes_on_rank_1'})
df_31 = df_31.drop(columns='rank')

df_32 = df_31.merge(df_rank_2, on=['bundesland', 'sexus', 'year'], how='left')
df_32 = df_32.rename(columns={'straftat': 'crimes_on_rank_2',
                               'certain_crime_percent_of_total_crime': 'percentage_of_rank_2_on_crime_total',
                               'schluessel': 'schluessel_crimes_on_rank_2'})
df_32 = df_32.drop(columns='rank')

df_33 = df_32.merge(df_rank_3, on=['bundesland', 'sexus', 'year'], how='left')
df_33 = df_33.rename(columns={'straftat': 'crimes_on_rank_3',
                               'certain_crime_percent_of_total_crime': 'percentage_of_rank_3_on_crime_total',
                               'schluessel': 'schluessel_crimes_on_rank_3'})
df_33 = df_33.drop(columns='rank')

df_34 = df_33.merge(df_rank_4, on=['bundesland', 'sexus', 'year'], how='left')
df_34 = df_34.rename(columns={'straftat': 'crimes_on_rank_4',
                               'certain_crime_percent_of_total_crime': 'percentage_of_rank_4_on_crime_total',
                               'schluessel': 'schluessel_crimes_on_rank_4'})
df_34 = df_34.drop(columns='rank')

df_35 = df_34.merge(df_rank_5, on=['bundesland', 'sexus', 'year'], how='left')
df_35 = df_35.rename(columns={'straftat': 'crimes_on_rank_5',
                               'certain_crime_percent_of_total_crime': 'percentage_of_rank_5_on_crime_total',
                               'schluessel': 'schluessel_crimes_on_rank_5'})
df_35 = df_35.drop(columns='rank')

df_laender_straftaten_insg_abs_u_rel_ranks_1_2_3_4_5 = df_35.copy() 
df_laender_straftaten_insg_abs_u_rel_ranks_1_2_3_4_5.head()

,bundesland,sexus,year,offenders,offenders_rel,schluessel_crimes_on_rank_1,crimes_on_rank_1,percentage_of_rank_1_on_crime_total,schluessel_crimes_on_rank_2,crimes_on_rank_2,percentage_of_rank_2_on_crime_total,schluessel_crimes_on_rank_3,crimes_on_rank_3,percentage_of_rank_3_on_crime_total,schluessel_crimes_on_rank_4,crimes_on_rank_4,percentage_of_rank_4_on_crime_total,schluessel_crimes_on_rank_5,crimes_on_rank_5,percentage_of_rank_5_on_crime_total
0,Baden-Württemberg,M,2018,10078,8600,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),31.78,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",20.91,*26*00,Ladendiebstahl insgesamt,13.97,674000,Sachbeschädigung §§ 303-305a StGB,9.49,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,3.15
1,Bayern,M,2018,10604,7998,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),36.55,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",20.71,674000,Sachbeschädigung §§ 303-305a StGB,11.11,*26*00,Ladendiebstahl insgesamt,9.64,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,3.27
2,Berlin,M,2018,3977,13427,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",23.21,*26*00,Ladendiebstahl insgesamt,18.63,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),16.29,674000,Sachbeschädigung §§ 303-305a StGB,12.42,210000,"Raub, räuberische Erpressung und räuberischer Angriff auf Kraftfahrer §§ 249-252, 255, 316a StGB",6.44
3,Brandenburg,M,2018,2469,11170,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),31.67,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",21.79,674000,Sachbeschädigung §§ 303-305a StGB,18.06,*26*00,Ladendiebstahl insgesamt,10.09,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,3.65
4,Bremen,M,2018,1125,16922,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",19.29,*26*00,Ladendiebstahl insgesamt,13.96,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),12.27,674000,Sachbeschädigung §§ 303-305a StGB,5.33,210000,"Raub, räuberische Erpressung und räuberischer Angriff auf Kraftfahrer §§ 249-252, 255, 316a StGB",5.24


### Calculate Growth Rate

In [10]:
df_51 = df_laender_abs.copy()
df_51 = df_51[["schluessel", "straftat", "bundesland", "year", 'sexus', "jugendliche_16_bis_unter_18"]]
df_51 = df_51[df_51["schluessel"] == "------"]
df_51 = df_51[["bundesland", "year", 'sexus', "jugendliche_16_bis_unter_18"]]

df_54 = pd.DataFrame(columns = ["bundesland", "year", 'sexus', "jugendliche_16_bis_unter_18", "growth_rate_abs"])

gender = ['X', 'W', 'M']
bundeslaender = ['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen',
       'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen',
       'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen',
       'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen']
for g in gender:
    df_52 = df_51[df_51["sexus"] == g]
    for b in bundeslaender:
        df_53 = df_52[df_52["bundesland"] == b]
        df_53["growth_rate_abs"] = df_53['jugendliche_16_bis_unter_18'].pct_change() * 100
        df_54 = pd.concat([df_54, df_53])

df_54 = df_54.drop(columns='jugendliche_16_bis_unter_18')

df_laender_growth_rate_abs = df_54.copy()
df_laender_growth_rate_abs.head(5)

/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_14752/1549961988.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_53["growth_rate_abs"] = df_53['jugendliche_16_bis_unter_18'].pct_change() * 100
/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_14752/1549961988.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_53["growth_rate_abs"] = df_53['jugendliche_16_bis_unter_18'].pct_change() * 100
/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_14752/1549961988.py:17: Setti

,bundesland,year,sexus,growth_rate_abs
32,Baden-Württemberg,2018,X,NaN
560,Baden-Württemberg,2019,X,-4.320753
1088,Baden-Württemberg,2020,X,-6.801587
1616,Baden-Württemberg,2021,X,-8.992591
2144,Baden-Württemberg,2022,X,15.720034


In [11]:
df_61 = df_laender_rel.copy()
df_61 = df_61[["schluessel", "straftat", "bundesland", "year", 'sexus', "jugendliche_16_bis_unter_18_rel"]]
df_61 = df_61[df_61["schluessel"] == "------"]
df_61 = df_61[["bundesland", "year", 'sexus', "jugendliche_16_bis_unter_18_rel"]]

df_64 = pd.DataFrame(columns = ["bundesland", "year", 'sexus', "jugendliche_16_bis_unter_18_rel", "growth_rate_rel"])

gender = ['X', 'W', 'M']
bundeslaender = ['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen',
       'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen',
       'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen',
       'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen']
for g in gender:
    df_62 = df_61[df_61["sexus"] == g]
    for b in bundeslaender:
        df_63 = df_62[df_62["bundesland"] == b]
        df_63["growth_rate_rel"] = df_63['jugendliche_16_bis_unter_18_rel'].pct_change() * 100
        df_64 = pd.concat([df_64, df_63])
        
df_64 = df_64.drop(columns='jugendliche_16_bis_unter_18_rel')

df_laender_growth_rate_rel = df_64.copy()
df_laender_growth_rate_rel.head(5)

/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_14752/3652961903.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_63["growth_rate_rel"] = df_63['jugendliche_16_bis_unter_18_rel'].pct_change() * 100
/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_14752/3652961903.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_63["growth_rate_rel"] = df_63['jugendliche_16_bis_unter_18_rel'].pct_change() * 100
/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_14752/3652961903.py:1

,bundesland,year,sexus,growth_rate_rel
32,Baden-Württemberg,2018,X,NaN
560,Baden-Württemberg,2019,X,-0.909043
1088,Baden-Württemberg,2020,X,-5.255775
1616,Baden-Württemberg,2021,X,-8.219895
2144,Baden-Württemberg,2022,X,16.230947


In [12]:
df_growth_rate_16_to_under_18 = df_laender_growth_rate_abs.merge(df_laender_growth_rate_rel, on=['bundesland', 'year', 'sexus'], how='left')
df_growth_rate_16_to_under_18.head()

,bundesland,year,sexus,growth_rate_abs,growth_rate_rel
0,Baden-Württemberg,2018,X,NaN,NaN
1,Baden-Württemberg,2019,X,-4.320753,-0.909043
2,Baden-Württemberg,2020,X,-6.801587,-5.255775
3,Baden-Württemberg,2021,X,-8.992591,-8.219895
4,Baden-Württemberg,2022,X,15.720034,16.230947


In [13]:
values_growth_rate_abs = df_growth_rate_16_to_under_18["growth_rate_abs"].tolist()
new_values_abs = []
for v in values_growth_rate_abs:
    if np.isnan(v):
        new_v = v
    else: 
        new_v = normal_round(v, 2)
    new_values_abs.append(new_v)
df_growth_rate_16_to_under_18["growth_rate_abs"] = new_values_abs

values_growth_rate_rel = df_growth_rate_16_to_under_18["growth_rate_rel"].tolist()
new_values_rel = []
for v in values_growth_rate_rel:
    if np.isnan(v):
        new_v = v
    else: 
        new_v = normal_round(v, 2)
    new_values_rel.append(new_v)
df_growth_rate_16_to_under_18["growth_rate_rel"] = new_values_rel

In [14]:
df_growth_rate_16_to_under_18.head()

,bundesland,year,sexus,growth_rate_abs,growth_rate_rel
0,Baden-Württemberg,2018,X,NaN,NaN
1,Baden-Württemberg,2019,X,-4.31,-0.90
2,Baden-Württemberg,2020,X,-6.79,-5.25
3,Baden-Württemberg,2021,X,-8.98,-8.21
4,Baden-Württemberg,2022,X,15.72,16.23


In [15]:
df_overview_fed_states_16_to_under_18 = df_laender_straftaten_insg_abs_u_rel_ranks_1_2_3_4_5.merge(df_growth_rate_16_to_under_18, on=['bundesland', 'sexus', 'year'], how='left')
#df_overview_fed_states = df_overview_fed_states.drop(columns='sexus')

df_overview_fed_states_16_to_under_18['age_group'] = 'jugendliche_16_bis_unter_18'

# bring in correct order
df_overview_fed_states_16_to_under_18 = df_overview_fed_states_16_to_under_18[['bundesland', 'year', 'sexus', 'age_group', 'offenders', 'offenders_rel',
       'growth_rate_abs', 'growth_rate_rel',
       'schluessel_crimes_on_rank_1', 'crimes_on_rank_1',
       'percentage_of_rank_1_on_crime_total', 'schluessel_crimes_on_rank_2',
       'crimes_on_rank_2', 'percentage_of_rank_2_on_crime_total',
       'schluessel_crimes_on_rank_3', 'crimes_on_rank_3',
       'percentage_of_rank_3_on_crime_total', 'schluessel_crimes_on_rank_4',
       'crimes_on_rank_4', 'percentage_of_rank_4_on_crime_total',
       'schluessel_crimes_on_rank_5', 'crimes_on_rank_5',
       'percentage_of_rank_5_on_crime_total']]

df_overview_fed_states_16_to_under_18.tail(5)

,bundesland,year,sexus,age_group,offenders,offenders_rel,growth_rate_abs,growth_rate_rel,schluessel_crimes_on_rank_1,crimes_on_rank_1,percentage_of_rank_1_on_crime_total,schluessel_crimes_on_rank_2,crimes_on_rank_2,percentage_of_rank_2_on_crime_total,schluessel_crimes_on_rank_3,crimes_on_rank_3,percentage_of_rank_3_on_crime_total,schluessel_crimes_on_rank_4,crimes_on_rank_4,percentage_of_rank_4_on_crime_total,schluessel_crimes_on_rank_5,crimes_on_rank_5,percentage_of_rank_5_on_crime_total
235,Saarland,2022,X,jugendliche_16_bis_unter_18,1238,7387,15.16,15.00,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",23.75,*26*00,Ladendiebstahl insgesamt,23.10,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),11.47,674000,Sachbeschädigung §§ 303-305a StGB,7.03,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,6.06
236,Sachsen,2022,X,jugendliche_16_bis_unter_18,4734,6972,19.18,17.40,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),17.19,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",16.41,*26*00,Ladendiebstahl insgesamt,14.39,674000,Sachbeschädigung §§ 303-305a StGB,12.27,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,7.65
237,Sachsen-Anhalt,2022,X,jugendliche_16_bis_unter_18,2649,7575,9.24,6.81,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",25.78,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),22.01,674000,Sachbeschädigung §§ 303-305a StGB,13.74,*26*00,Ladendiebstahl insgesamt,13.48,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,5.62
238,Schleswig-Holstein,2022,X,jugendliche_16_bis_unter_18,3383,6197,6.82,8.47,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",20.75,*26*00,Ladendiebstahl insgesamt,19.80,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),19.33,674000,Sachbeschädigung §§ 303-305a StGB,9.02,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,6.38
239,Thüringen,2022,X,jugendliche_16_bis_unter_18,2314,6601,8.69,8.33,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),22.21,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",19.53,*26*00,Ladendiebstahl insgesamt,13.79,674000,Sachbeschädigung §§ 303-305a StGB,13.35,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,7.69


In [16]:
df_overview_fed_states_16_to_under_18 = df_overview_fed_states_16_to_under_18.fillna('n.a.')

In [17]:
df_overview_fed_states_16_to_under_18.head()

,bundesland,year,sexus,age_group,offenders,offenders_rel,growth_rate_abs,growth_rate_rel,schluessel_crimes_on_rank_1,crimes_on_rank_1,percentage_of_rank_1_on_crime_total,schluessel_crimes_on_rank_2,crimes_on_rank_2,percentage_of_rank_2_on_crime_total,schluessel_crimes_on_rank_3,crimes_on_rank_3,percentage_of_rank_3_on_crime_total,schluessel_crimes_on_rank_4,crimes_on_rank_4,percentage_of_rank_4_on_crime_total,schluessel_crimes_on_rank_5,crimes_on_rank_5,percentage_of_rank_5_on_crime_total
0,Baden-Württemberg,2018,M,jugendliche_16_bis_unter_18,10078,8600,n.a.,n.a.,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),31.78,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",20.91,*26*00,Ladendiebstahl insgesamt,13.97,674000,Sachbeschädigung §§ 303-305a StGB,9.49,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,3.15
1,Bayern,2018,M,jugendliche_16_bis_unter_18,10604,7998,n.a.,n.a.,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),36.55,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",20.71,674000,Sachbeschädigung §§ 303-305a StGB,11.11,*26*00,Ladendiebstahl insgesamt,9.64,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,3.27
2,Berlin,2018,M,jugendliche_16_bis_unter_18,3977,13427,n.a.,n.a.,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",23.21,*26*00,Ladendiebstahl insgesamt,18.63,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),16.29,674000,Sachbeschädigung §§ 303-305a StGB,12.42,210000,"Raub, räuberische Erpressung und räuberischer Angriff auf Kraftfahrer §§ 249-252, 255, 316a StGB",6.44
3,Brandenburg,2018,M,jugendliche_16_bis_unter_18,2469,11170,n.a.,n.a.,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),31.67,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",21.79,674000,Sachbeschädigung §§ 303-305a StGB,18.06,*26*00,Ladendiebstahl insgesamt,10.09,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,3.65
4,Bremen,2018,M,jugendliche_16_bis_unter_18,1125,16922,n.a.,n.a.,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",19.29,*26*00,Ladendiebstahl insgesamt,13.96,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),12.27,674000,Sachbeschädigung §§ 303-305a StGB,5.33,210000,"Raub, räuberische Erpressung und räuberischer Angriff auf Kraftfahrer §§ 249-252, 255, 316a StGB",5.24


# Now Create same DataFrame for other age groups and then concat them to df_overview_fed_states

In [18]:
%store df_overview_fed_states_16_to_under_18

Stored 'df_overview_fed_states_16_to_under_18' (DataFrame)
